<a href="https://colab.research.google.com/github/SupriyaKnarayan/Flight_fare_price_detection/blob/main/Project_1_Flight_Fare_price_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**
Flight ticket prices is something hard to guess, today we might see a price,check out the price of the same flight tomorrow, it will be a different story.
We might have often heard travellers saying that flight ticket prices are so unpredictable.

Here you will be provided with the prices of flight tickets for various airlines between the month of March and June  of 2019and between various cities. Size of training dataset :10683 records Size of test dataset:2671 records. Features: Airline:The name of the airline.

Date_of_Journey:The date of the journey Source:The Source from which the service begins.Destination:The Destination where the service ends.Route:The Route taken by  the Flight to reach the destination. Dep_time: The Time when the journey Starts from the source.
Arrival_Time:Time of Arrival at the destination.Duration:Total duration OF flight.Total_Stops:Total Stops Between the source and destination.


# **Problem Statement**

Build a Machine Learning Model to predict Flight ticket fare for different airline.

# **Loading the Data**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNet, Lasso,Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import datetime
from mlxtend.regressor import StackingCVRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# Data Cleaning & Preprocessing


In [3]:
from google.colab import files
uploaded = files.upload()

Saving Data_Train.xlsx to Data_Train (1).xlsx


In [4]:
from google.colab import files
uploaded = files.upload()

Saving Test_set.xlsx to Test_set (1).xlsx


In [5]:
import pandas as pd

train = pd.read_excel('/content/Data_Train.xlsx')
test=pd.read_excel('/content/Test_set.xlsx')

In [6]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [8]:
len_train=train.shape[0]

In [9]:
for i in train.columns:
    print("Unique values in", i, train[i].nunique())

Unique values in Airline 12
Unique values in Date_of_Journey 44
Unique values in Source 5
Unique values in Destination 6
Unique values in Route 128
Unique values in Dep_Time 222
Unique values in Arrival_Time 1343
Unique values in Duration 368
Unique values in Total_Stops 5
Unique values in Additional_Info 10
Unique values in Price 1870


In [10]:
for i in test.columns:
    print("Unique values in", i, train[i].nunique())

Unique values in Airline 12
Unique values in Date_of_Journey 44
Unique values in Source 5
Unique values in Destination 6
Unique values in Route 128
Unique values in Dep_Time 222
Unique values in Arrival_Time 1343
Unique values in Duration 368
Unique values in Total_Stops 5
Unique values in Additional_Info 10


In [28]:
train = pd.concat([train, test], ignore_index=True)
train = train.reset_index(drop=True)